# Grain Level Stress and Strain Analysis
This notebook performs an analysis of simulated microstructures, combining stress/strain values for each voxel into values for each node (grain). The stress and strain values for each grain are fitted to a Generalized Extreme Value (GEV) distribution, and the parameters of these distributions are appended to the feature data.

In [36]:
import pandas as pd
from scipy.stats import genextreme
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew

## Load and Preprocess the Data
The data includes two files: one that contains the grain level data, and another that contains the voxel level data. Each feature ID in the grain data corresponds to a grain, and each grain contains multiple voxels.

In [37]:
NUM_MICROSTRUCTURES_START = 0 # Including start
NUM_MICROSTRUCTURES_END = 1 # Not including end

In [38]:
def load_data(microstructure_num):
    # Load the cell data
    cell_data = pd.read_csv(f'CellData_FakeMatl_{microstructure_num}.csv')

    # Load the feature data
    # Read the first line of the file to get the number of grains
    with open(f'FeatureData_FakeMatl_{microstructure_num}.csv', 'r') as f:
        num_grains = int(f.readline())

    # Read the CSV file skipping the first line and only reading the number of lines equal to the number of grains
    feature_data_part1 = pd.read_csv(f'FeatureData_FakeMatl_{microstructure_num}.csv', skiprows=1, nrows=num_grains)

    # Strip leading and trailing spaces from column names
    cell_data.columns = cell_data.columns.str.strip()

    return cell_data, feature_data_part1

## Calculate Mean Stress and Strain
We calculate the mean stress and strain values for each grain. We consider columns starting with 'E' in the cell data as strain, and columns starting with 'S' as stress.

In [39]:
def calculate_mean_strain_stress(cell_data):
    # Identify the stress and strain columns
    strain_cols = [col for col in cell_data.columns if col.startswith('E') and len(col) == 3]
    stress_cols = [col for col in cell_data.columns if col.startswith('S') and len(col) == 3]

    # Calculate the mean stress and strain for each grain
    mean_strain = cell_data.groupby('FeatureIds')[strain_cols].mean()
    mean_stress = cell_data.groupby('FeatureIds')[stress_cols].mean()

    return mean_strain, mean_stress, strain_cols, stress_cols

## Fit GEV Distribution and Extract Parameters
We fit a Generalized Extreme Value (GEV) distribution to these mean values and extract the distribution parameters (location, scale, and shape). We then append these parameters to the feature data.

The Generalized Extreme Value (GEV) distribution is a family of continuous probability distributions developed within extreme value theory to combine the Gumbel, Fr\Ă©chet and Weibull families also known as type I, II and III extreme value distributions. The GEV distribution is often used to model the maxima of other distributions.

The GEV distribution is defined as:

$$
F(x;\mu,\sigma,\xi) = \exp \left\{ - \left[ 1 + \xi \left( \frac{x - \mu}{\sigma} \right) \right]^{-1/\xi} \right\}, \quad \text{for } \xi \neq 0
$$

$$
F(x;\mu,\sigma,\xi) = \exp \left\{ - \exp \left[ - \left( \frac{x - \mu}{\sigma} \right) \right] \right\}, \quad \text{for } \xi = 0
$$

where $\mu$ is the location parameter, $ \sigma > 0 $ is the scale parameter, and $\xi$ is the shape parameter. The case where $\xi = 0$ corresponds to the Gumbel family.

We are using the method of moments to estimate these parameters. This method sets the moments of the theoretical distribution equal to the moments of the empirical distribution, and solves for the parameters of the theoretical distribution. For the GEV distribution, the method of moments estimates are:

- Shape parameter $\xi$: $0.5 - \text{sample skewness} / 6$
- Location parameter $\mu$: $\text{sample mean}$
- Scale parameter $\sigma$: $\text{sample standard deviation} * \(\sqrt{6}\) / \(\pi\)$

In [80]:
# Function to estimate GEV parameters
def estimate_gev_parameters(series):
    # shape = 0.5 - skew(series) / 6
    # loc = series.mean()
    # scale = series.std() * (6**0.5) / np.pi
    # return shape, loc, scale
    if len(series) == 1:
        return None, series.iloc[0], None
    shape, loc, scale = genextreme.fit(series)
    return shape, loc, scale


def gev_distribution(cell_data, feature_data_part1, mean_strain, mean_stress, strain_cols, stress_cols):
    # Initialize dataframes to hold the GEV parameters
    gev_params_strain = pd.DataFrame(index=mean_strain.index,
                                     columns=[f"{col}_{param}" for col in strain_cols for param in ['shape', 'loc', 'scale']],
                                     dtype=float)
    gev_params_stress = pd.DataFrame(index=mean_stress.index,
                                     columns=[f"{col}_{param}" for col in stress_cols for param in ['shape', 'loc', 'scale']],
                                     dtype=float)

    # Estimate the GEV parameters for each grain
    print(mean_strain.index)
    print(cell_data)
    for grain_id in mean_strain.index:
        for col in strain_cols:
            print("Grain: ", grain_id, "Parameter: ", col)
            params = estimate_gev_parameters(cell_data[cell_data['FeatureIds'] == grain_id][col].dropna())
            for param, value in zip(['shape', 'loc', 'scale'], params):
                gev_params_strain.loc[grain_id, f"{col}_{param}"] = value

    for grain_id in mean_stress.index:
        for col in stress_cols:
            print("Grain: ", grain_id, "Parameter: ", col)
            params = estimate_gev_parameters(cell_data[cell_data['FeatureIds'] == grain_id][col].dropna())
            for param, value in zip(['shape', 'loc', 'scale'], params):
                gev_params_stress.loc[grain_id, f"{col}_{param}"] = value

    # Concatenate the GEV parameter dataframes with the feature data
    feature_data_part1['Feature_ID'] = feature_data_part1['Feature_ID'].astype(int)
    feature_data_part1_extended = pd.merge(feature_data_part1, gev_params_strain, left_on='Feature_ID', right_index=True)
    feature_data_part1_extended = pd.merge(feature_data_part1_extended, gev_params_stress, left_on='Feature_ID', right_index=True)

    return feature_data_part1_extended

## Visualize the Results
We create histograms of the GEV parameters for each stress variable to visualize their distributions.

In [81]:
%matplotlib inline
def visualize_value_distribution(feature_data_part1_extended, stress_cols):
    # Create a list of parameters
    params = ['shape', 'loc', 'scale']
    print(stress_cols)
    # Create plots for each stress variable
    for col in stress_cols:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        fig.suptitle(f'GEV Parameters for {col}', fontsize=16)

        for i, param in enumerate(params):
            axs[i].hist(feature_data_part1_extended[f"{col}_{param}"].dropna(), bins=30, edgecolor='k')
            axs[i].set_title(f'{param.capitalize()}')
            axs[i].set_xlabel(f'{param.capitalize()} Value')
            axs[i].set_ylabel('Frequency')

        plt.tight_layout()
        plt.subplots_adjust(top=0.88)
        plt.show()

In [82]:
from scipy.stats import genextreme
import matplotlib.cm as cm

def visualize_gev_distribution(feature_data_part1_extended, stress_cols, strain_cols):
    # GEV line plots
    for col in stress_cols + strain_cols:
        plt.figure(figsize=(10, 6))
        plt.title(f'Fitted GEV Distribution for {col}')

        # Create a colormap
        grain_ids = feature_data_part1_extended.index.unique()
        colors = cm.rainbow(np.linspace(0, 1, len(grain_ids)))

        for grain_id, color in zip(grain_ids, colors):
            shape = feature_data_part1_extended.loc[grain_id, f'{col}_shape']
            loc = feature_data_part1_extended.loc[grain_id, f'{col}_loc']
            scale = feature_data_part1_extended.loc[grain_id, f'{col}_scale']
            x = np.linspace(genextreme.ppf(0.01, shape, loc, scale), genextreme.ppf(0.99, shape, loc, scale), 100)
            plt.plot(x, genextreme.pdf(x, shape, loc, scale), color=color)

        plt.xlabel(f'{col} Value')
        plt.ylabel('Probability Density')

        # Create a colorbar
        sm = plt.cm.ScalarMappable(cmap=cm.rainbow, norm=plt.Normalize(vmin=min(grain_ids), vmax=max(grain_ids)))
        plt.colorbar(sm, label='Grain ID')

        plt.show()

## Save the Results
Finally, we save the extended feature data to a new CSV file.

In [83]:
# Save the extended feature data to a new CSV file
def save_extended_feature_data(feature_data_part1_extended, microstructure_num):
    feature_data_part1_extended.to_csv(f'ExtendedFeatureData_FakeMatl_{microstructure_num}.csv', index=False)
    print(f"Saved extended feature data for microstructure {microstructure_num}")

In [85]:
# Run through all microstructures
for microstructure_num in range(NUM_MICROSTRUCTURES_START, NUM_MICROSTRUCTURES_END):
    # Load the data
    cell_data, feature_data_part1 = load_data(microstructure_num)
    mean_strain, mean_stress, strain_cols, stress_cols = calculate_mean_strain_stress(cell_data)

    # Fit the GEV distribution
    feature_data_part1_extended = gev_distribution(cell_data, feature_data_part1, mean_strain, mean_stress, strain_cols, stress_cols)

    # Visualize the results
    # visualize_value_distribution(feature_data_part1_extended, stress_cols)
    # visualize_gev_distribution(feature_data_part1_extended, stress_cols, strain_cols)

    # Save the results
    save_extended_feature_data(feature_data_part1_extended, microstructure_num)

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       237, 238, 239, 240, 241, 242, 243, 244, 245, 246],
      dtype='int64', name='FeatureIds', length=246)
            E11       E12       E13       E22       E23       E33         S11  \
0     -0.068004  0.001799  0.023700  0.029455 -0.005069 -0.044397 -805.019600   
1     -0.065395 -0.002569  0.022720  0.021885 -0.002614 -0.035775 -527.208000   
2     -0.047230 -0.000174  0.025682  0.009990 -0.003698 -0.040860 -271.473330   
3     -0.044112  0.002064  0.022896  0.007844 -0.000094 -0.039198  -18.032635   
4     -0.039601 -0.000327 -0.002366  0.011405  0.003115 -0.044364   73.482086   
...         ...       ...       ...       ...       ...       ...         ...   
24384 -0.044378 -0.012461 -0.005976  0.020697  0.002179 -0.051742   58.527855   
24385 -0.053875 -0.011222  0.002242  0.024468 -0.000957 -0.045608   54.428158   
24386 -0.053358 -0.008606  0.001509  0.024129 -0.002813 -0.045574   78.734730   
24387 -0.051

# Alternative: Fit GEV Distribution using Maximum Likelihood Estimation
As an alternative to the method of moments, we can use the Maximum Likelihood Estimation (MLE) method to fit the GEV distribution and extract the parameters. This method often provides better estimates, but it requires numerical optimization and is more computationally intensive.

Please note that the following code can be computationally intensive and may take a while to run, especially with large datasets. You should only run it if you have sufficient computational resources.

In [9]:
# The following code is commented out due to its high computational demand. 
# Uncomment it if you want to run it on your local system with sufficient computational resources.

# Initialize dataframes to hold the GEV parameters
# gev_params_strain_mle = pd.DataFrame(index=mean_strain.index, 
#                                  columns=[f"{col}_{param}" for col in strain_cols for param in ['shape', 'loc', 'scale']],
#                                  dtype=float)
# gev_params_stress_mle = pd.DataFrame(index=mean_stress.index, 
#                                  columns=[f"{col}_{param}" for col in stress_cols for param in ['shape', 'loc', 'scale']],
#                                  dtype=float)

# Fit a GEV distribution to the mean values and extract the parameters using MLE
# for grain_id in mean_strain.index:
#    for col in strain_cols:
#        params = genextreme.fit(cell_data[cell_data['FeatureIds'] == grain_id][col].dropna())
#        for param, value in zip(['shape', 'loc', 'scale'], params):
#            gev_params_strain_mle.loc[grain_id, f"{col}_{param}"] = value

# for grain_id in mean_stress.index:
#    for col in stress_cols:
#        params = genextreme.fit(cell_data[cell_data['FeatureIds'] == grain_id][col].dropna())
#        for param, value in zip(['shape', 'loc', 'scale'], params):
#            gev_params_stress_mle.loc[grain_id, f"{col}_{param}"] = value

# Concatenate the GEV parameter dataframes with the feature data
# feature_data_part1_extended_mle = pd.merge(feature_data_part1_reset, gev_params_strain_mle, left_on='Feature_ID', right_index=True)
# feature_data_part1_extended_mle = pd.merge(feature_data_part1_extended_mle, gev_params_stress_mle, left_on='Feature_ID', right_index=True)